# Yellowdig to Blackboard Converter

Description : 
     This notebook processes Yellowdig participation grades from a CSV export 
     and formats them for direct import into Blackboard's grade center.
Output:
     A CSV file ready to be imported into Blackboard.

@author Matt Thomas with ChatGPT tweaks 

Created on May  6, 2025

In [ ]:
# Imports
import pandas as pd
from pathlib import Path

## Edit for your Course

In [ ]:
# Edit these parameters for your run
input_csv = Path("Yellowdig-points.csv")   # e.g., 'Yellowdig-points.csv'
possible_course_points = 1000              # e.g., total course points from syllabus
percent_yellowdig = 2                      # e.g., Yellowdig share of grade as INTEGER percent

# Output filename (you can change this if desired)
BLACKBOARD_FILE_NAME = "ReadyToImport_Yellowdig.csv"

## Run Program 

In [ ]:
# Processing logic
points = possible_course_points * (percent_yellowdig / 100)

# Load data
df = pd.read_csv(input_csv)

# Remove unneeded columns (if present)
cols_to_drop = [c for c in ['user/username', 'Total Points Earned'] if c in df.columns]
df = df.drop(columns=cols_to_drop, errors='ignore')

# Add Yellowdig score column
if 'Participation Grade' not in df.columns:
    raise KeyError("Expected 'Participation Grade' column in the input CSV.")
df['Yellowdig'] = (df['Participation Grade'] * points).round(4)

# Clean-up column names to match Blackboard requirements
rename_map = {'user/firstname': 'First Name', 'user/lastname': 'Last Name'}
df = df.rename(columns=rename_map)
if 'Participation Grade' in df.columns:
    df = df.drop(columns=['Participation Grade'])

# Derive Username from email
if 'user/primary-email' not in df.columns:
    raise KeyError("Expected 'user/primary-email' column in the input CSV.")
df['Username'] = df['user/primary-email'].astype(str).str.split('@').str[0]

# Final Blackboard format
needed = ['Username', 'Last Name', 'First Name', 'Yellowdig']
missing = [c for c in ['Last Name', 'First Name'] if c not in df.columns]
if missing:
    raise KeyError(f"Missing expected columns: {missing}. Check the input CSV headers.")
final_df = df[needed]

# Write file
final_df.to_csv(BLACKBOARD_FILE_NAME, index=False)
print(f"Wrote: {BLACKBOARD_FILE_NAME}")

## Preview of Output

In [ ]:
# Optional: preview the first few rows of the output if it exists
out = Path(BLACKBOARD_FILE_NAME)
if out.exists():
    display(pd.read_csv(out).head())
else:
    print("Run the processing cell above to generate the output first.")